In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, LSTM)
from keras.models import Model, load_model
from PIL import Image
import numpy as np
from pydub import AudioSegment
from sklearn.metrics import confusion_matrix

In [ ]:
genres_musicaux = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [ ]:
def get_features():
  labels = []
  features = []
  i = 0
  for g in genres_musicaux:
    print(g)
    for filename in os.listdir(os.path.join('/content/gdrive/MyDrive/Projet Majeure/Premier test/audio3sec/',f'{g}')):
      if not filename.startswith('.'):    #avoiding the .DS_Store file
        path = os.path.join('/content/gdrive/MyDrive/Projet Majeure/Premier test/audio3sec/',f'{g}',f'{filename}')
        y,sr = librosa.load(path,duration=3)
        mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr, n_fft=1012, hop_length=256, n_mfcc=20))

        if len(mfcc[1]) != 259 :      #avoiding some problem of array length
          diff = 259-len(mfcc[1])
          zeros = np.zeros((20,diff))
          mfcc = np.concatenate((mfcc,zeros),axis = 1)
          print(mfcc.shape)
        features.append(mfcc)
        labels.append(i)
    i += 1
  return(np.array(features), np.array(labels))

In [ ]:
X,y = get_features()

blues
classical
(20, 259)
country
(20, 259)
disco
(20, 259)
hiphop
(20, 259)
jazz
metal
pop
reggae
rock


In [ ]:
print(X.shape)
print(y.shape)

(9900, 20, 259)
(9900,)


In [ ]:
#Split twice to get the validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
#Print the shapes
X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val)

((5568, 20, 259), (2475, 20, 259), (1857, 20, 259), 5568, 2475, 1857)

In [ ]:
X_train = tensorflow.expand_dims(X_train, axis = -1)     #expansion for convolution
X_test = tensorflow.expand_dims(X_test, axis = -1)     #expansion for convolution
X_val = tensorflow.expand_dims(X_val, axis = -1)     #expansion for convolution

print('Expand_dims shape', X_train.shape, X_test.shape, X_val.shape)

Expand_dims shape (5568, 20, 259, 1) (2475, 20, 259, 1) (1857, 20, 259, 1)


In [ ]:
input_shape=(20,259,1)

In [ ]:
model = keras.Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides = (1,1),activation='relu', input_shape=input_shape))
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,kernel_size=(3,3),strides = (1,1),activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128,kernel_size=(3,3),strides = (1,1),activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Flatten())
model.add(Dropout(rate=0.3))
model.add(Dense(10, activation='softmax'))


In [ ]:
model.compile(optimizer='rmsprop',loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size = 300, epochs=30, validation_data= (X_val, y_val), )

Epoch 1/30
19/19 [==============================] - 52s 3s/step - loss: 2.0358 - accuracy: 0.4120 - val_loss: 16.2544 - val_accuracy: 0.0980
Epoch 2/30
19/19 [==============================] - 51s 3s/step - loss: 1.2730 - accuracy: 0.5903 - val_loss: 10.1410 - val_accuracy: 0.1228
Epoch 3/30
19/19 [==============================] - 51s 3s/step - loss: 0.9456 - accuracy: 0.6893 - val_loss: 6.6652 - val_accuracy: 0.1427
Epoch 4/30
19/19 [==============================] - 49s 3s/step - loss: 0.7594 - accuracy: 0.7360 - val_loss: 3.6546 - val_accuracy: 0.2380
Epoch 5/30
19/19 [==============================] - 49s 3s/step - loss: 0.5794 - accuracy: 0.8021 - val_loss: 2.4009 - val_accuracy: 0.3560
Epoch 6/30
19/19 [==============================] - 49s 3s/step - loss: 0.5355 - accuracy: 0.8141 - val_loss: 2.0651 - val_accuracy: 0.4184
Epoch 7/30
19/19 [==============================] - 54s 3s/step - loss: 0.4052 - accuracy: 0.8608 - val_loss: 2.2466 - val_accuracy: 0.3861
Epoch 8/30
19/19 [

In [ ]:
y_pred=model.predict(X_test)
print('Confusion_matrix: ',tensorflow.math.confusion_matrix(y_test, np.argmax(y_pred,axis=1)))

78/78 [==============================] - 6s 70ms/step
Confusion_matrix:  tf.Tensor(
[[164   2  40   9   1   8   0   1   4  18]
 [  1 242   4   0   0   1   0   0   0   0]
 [  0   4 213   1   0   6   0   9   1  14]
 [  3   1  20 167   0   4   0   8  13  32]
 [  8   1   4  33 143   0   0  11  36  11]
 [  3  25  24   1   0 178   0   3   1  12]
 [  5   0   8  21   3   2 151   1   4  53]
 [  2   3  14   6   6   2   0 200   6   8]
 [  5   0  28  24  11   0   0  11 158  10]
 [  5   3  58  11   0   4   2   8   7 150]], shape=(10, 10), dtype=int32)


### Test avec musiques

In [ ]:
StrtMin = 1
StrtSec = 0

EndMin = 1
EndSec = 40

# Time to milliseconds conversion
StrtTime = StrtMin*60*1000+StrtSec*1000
EndTime = StrtMin*60*1000+EndSec*1000

In [ ]:
audio = AudioSegment.from_wav('/content/DEATH DECLINE - Useless Sacrifice [Brutal Death Metal _ Thrash Metal]-IaJ2UHiTa0o.wav')
newAudio = audio[StrtTime:EndTime]
newAudio.export('/content/song.wav', format='wav')

for w in range(0,10):
          t1 = 3*(w)*1000
          t2 = 3*(w+1)*1000
          newAudio = AudioSegment.from_wav('/content/song.wav')
          new = newAudio[t1:t2]
          new.export(f'/content/{str(w)}.wav', format='wav')


In [ ]:
classe = []

for i in range(0,10):
  y,sr = librosa.load(f'{str(w)}.wav',duration=3)
  data = np.array([librosa.feature.mfcc(y=y,sr=sr, n_fft=1012, hop_length=256, n_mfcc=20)])


  data = tensorflow.expand_dims(data, axis = -1)
  prediction = model.predict(data)
  class_label = np.argmax(prediction)
  print(class_label)
  classe.append(class_label)

moyenne = int(sum(classe)/len(classe))
print(genres_musicaux[moyenne])

1/1 [==============================] - 0s 34ms/step
4
1/1 [==============================] - 0s 51ms/step
4
1/1 [==============================] - 0s 53ms/step
4
1/1 [==============================] - 0s 36ms/step
4
1/1 [==============================] - 0s 34ms/step
4
1/1 [==============================] - 0s 33ms/step
4
1/1 [==============================] - 0s 34ms/step
4
1/1 [==============================] - 0s 35ms/step
4
1/1 [==============================] - 0s 35ms/step
4
1/1 [==============================] - 0s 37ms/step
4
hiphop


In [ ]:
model.save('/content/gdrive/MyDrive/MFCCTunningSpectro')